In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import fastai
from fastai.vision import *
from pathlib import Path
from skimage.io import imread,imshow,imsave
import warnings
from collections import defaultdict
import pandas as pd
import PIL
from PIL import Image
from skimage.io import imread,imsave,imshow

In [3]:
import pickle
from tqdm import tqdm_notebook  as tqdm
import warnings

In [4]:
path_img = Path('train_images')
lbl_dir = Path('train_labels')
tst_dir = Path('test_images')
mixup_dir = Path('mixup_data_trn')
mixup_lbl = Path('mixup_labels')

In [5]:
trn_imgs = path_img.ls()

In [6]:
len(trn_imgs)

12569

In [7]:
_ = [im_fl for im_fl in trn_imgs if im_fl.is_file()]

In [8]:
len(_)

12568

In [9]:
trn_imgs = _

## get distirbution of defects

In [10]:
df = pd.read_csv('train.csv')

In [11]:
def f():
    return []

In [12]:
imgs_2_defs = defaultdict(f)

In [13]:
#for fl in tqdm(trn_imgs):
#    fn = fl.name
#    for i in [1,2,3,4]:
#        if not pd.isna(df[df.ImageId_ClassId == (fn+'_'+str(i))]['EncodedPixels']).values[0]:
#            imgs_2_defs[fn].append(i)

In [17]:
len(imgs_2_defs)

6666

 6666-defect,5902-non defective

In [16]:
#fl = open('defects_dict.pkl',mode='wb')
#pickle.dump(imgs_2_defs,fl)

In [15]:
with open('defects_dict.pkl',mode='rb') as fl:
    imgs_2_defs = pickle.load(fl)

In [18]:
defects_count = defaultdict(lambda:0)

In [19]:
for fn,defs in tqdm(imgs_2_defs.items()):
    string = ""
    for i in defs:
        string += ('_'+str(i))
    defects_count[string] += 1

In [20]:
defects_count

defaultdict(<function __main__.<lambda>()>,
            {'_3': 4759,
             '_1': 769,
             '_4': 516,
             '_2': 195,
             '_3_4': 284,
             '_1_3': 91,
             '_1_2': 35,
             '_2_3': 14,
             '_2_4': 1,
             '_1_2_3': 2})

 defects are {'_3': 4759,
             '_1': 769,
             '_4': 516,
             '_2': 195,
             '_3_4': 284,
             '_1_3': 91,
             '_1_2': 35,
             '_2_3': 14,
             '_2_4': 1,
             '_1_2_3': 2})

## combine images

In [21]:
#defects to images mapping
def_2_imgs = defaultdict(f)

go through all the files and add missing defect type, these defects are taken from files that have only one defect.

In [22]:
for fn,defs in tqdm(imgs_2_defs.items()):
    if len(defs) == 1:
        def_2_imgs[defs[0]].append(fn)

In [23]:
#iterate through trn images and go on adding missing defect
#    1) iterate through trn images
#    2) open trn image, one its mask
#    3) unique on mask, find missing defect
#    4) get a image file,mask that has missing defect
#    5) check if the defect we add covers(intersects) the esisting defects
#    5) if yes, get a new missing defect file,mask until no intersection is found
#    6) change mask to boolean 2d array and index into original image, defect image and equate them to transfer defect

In [23]:
# random sequences for getting defect 1,2,3,4 images
def_1_rnd_indx = np.arange(len(def_2_imgs[1]))
def_2_rnd_indx = np.arange(len(def_2_imgs[2]))
def_3_rnd_indx = np.arange(len(def_2_imgs[3]))
def_4_rnd_indx = np.arange(len(def_2_imgs[4]))
np.random.shuffle(def_1_rnd_indx)
np.random.shuffle(def_2_rnd_indx)
np.random.shuffle(def_3_rnd_indx)
np.random.shuffle(def_4_rnd_indx)
# array of shuffeled indexes, with lenght as per no of defect files
rnd_dfct_idxs = [def_1_rnd_indx,
                 def_2_rnd_indx,
                 def_3_rnd_indx,
                 def_4_rnd_indx]

In [25]:
# generator of file and msk paths based on linear index, initialise with defect type and keep receiving defect files
def get_img_from_defct_type(def_type:int):
    # init indexes and limits of indexes for defect files names array
    ind_defects = [0,0,0,0]
    ind_defects_lms = [768,194,4758,515]
    # convert linear index to random
    while True:
        idx  = ind_defects[def_type-1]        # linear idx, has to change after use
        #print(idx,len(rnd_dfct_idxs[def_type-1]))
        idxr = rnd_dfct_idxs[def_type-1][idx] # get random idx from pre shuffled array
        fn = def_2_imgs[def_type][idxr]     # get name using rnd idx
        img_pth = path_img/fn
        msk_pth = lbl_dir/(fn[:-4]+'.png')
        # update idx values
        ind_defects[def_type-1] += 1
        if ind_defects[def_type-1] > ind_defects_lms[def_type-1]:
            ind_defects[def_type-1] = 0
        yield img_pth,msk_pth
        

In [26]:
# defect img,msk generators
d1 = get_img_from_defct_type(1)
d2 = get_img_from_defct_type(2)
d3 = get_img_from_defct_type(3)
d4 = get_img_from_defct_type(4)
d = [d1,d2,d3,d4]

In [27]:
warnings.filterwarnings('ignore')

In [28]:
len(trn_imgs)

12568

In [29]:
ind_defects_lms = [768,194,4758,515]
# run only on images with atleast one defect
for fn in tqdm(imgs_2_defs.keys()):
    # read img, mask
    fn = path_img/fn      #string to proper path
    im1 = imread(fn,as_gray=True)
    m1  = imread(lbl_dir/(fn.stem+'.png'))
    # find defs in img
    defs = np.unique(m1)
    # only care about imgs with atleast one defect
    if len(defs) >=2:
        #iterate to get un-present defects
        for i in [1,2,3,4]:
            if i not in defs:
                loop_cnt = 0
                #iterate to get non intersecting defect file
                while True:
                    #print('ok')
                    loop_cnt += 1
                    fn,mn = next(d[i-1])
                    im2 = imread(fn,as_gray=True)
                    m2  = imread(mn)
                    m1b = m1.astype(bool)
                    m2b = m2.astype(bool)
                    if (m1b.sum()+m2b.sum() == (m1b+m2b).sum()) or loop_cnt>ind_defects_lms[i-1]: # equals means intersection = 0
                        break # stop iterating if intersection is zero
                if not loop_cnt>ind_defects_lms[i-1]:
                    im1[m2b] = im2[m2b] # transfer defect
                    m1[m2b] = m2[m2b]
        imsave(mixup_dir/(fn.stem+'.png'),im1)
        imsave(mixup_lbl/(fn.stem+'.png'),m1)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

...

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

## distribution of generated dataset

In [30]:
imgs = [fl for fl in mixup_lbl.ls() if fl.is_file()]

In [31]:
len(imgs)

1960

In [32]:
distri = defaultdict(lambda:[])

for fl in tqdm(imgs):
    msk = imread(fl)
    defs = np.unique(msk)
    string = ''
    for d in defs:
        if not d==0:
            string += ('_'+str(d))
    distri[string].append(fl)

In [33]:
distri.keys()

dict_keys(['_1_2_3_4', '_1_2_3', '_1_2_4', '_1_3'])

In [34]:
len(distri['_1_2_3_4']),len(distri['_1_2_3']),len(distri['_1_2_4']),len(distri['_1_3'])

(1954, 4, 1, 1)